In [12]:
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',30)

#获取数据
data = pd.read_csv("ad_performance.csv",index_col=0)

# 对数据审查,是否有缺失值
print('{:*^60}'.format('数据样本：统计描述'))
print(data.describe().round(4).T)

# 对缺失值的填充(均值)
print('{:*^60}'.format('缺失值：均值填充法'))
# print(data[data.isna().values])
data = data.fillna(419.77)

*************************数据样本：统计描述**************************
        count      mean        std   min       25%       50%       75%  \
日均UV    889.0  540.8468  1634.4105  0.06    6.1800  114.1800  466.8700   
平均注册率   889.0    0.0014     0.0033  0.00    0.0000    0.0000    0.0014   
平均搜索量   889.0    0.0305     0.1062  0.00    0.0006    0.0032    0.0118   
访问深度    889.0    2.1672     3.8005  1.00    1.3923    1.7931    2.2162   
平均停留时间  887.0  262.6692   224.3649  1.64  126.0200  236.5500  357.9850   
订单转化率   889.0    0.0029     0.0116  0.00    0.0000    0.0002    0.0020   
投放总时间   889.0   16.0529     8.5094  1.00    9.0000   16.0000   24.0000   

               max  
日均UV    25294.7700  
平均注册率       0.0391  
平均搜索量       1.0370  
访问深度       98.9799  
平均停留时间   4450.8300  
订单转化率       0.2165  
投放总时间      30.0000  
*************************缺失值：均值填充法**************************
     渠道代号  日均UV  平均注册率  平均搜索量  访问深度  平均停留时间  订单转化率  投放总时间 素材类型 广告类型 合作方式  \
323  A347  1.53    0.0    0.0   1.0     N

In [20]:
# 计算，合并:相关性
print('{:*^60}'.format('计算相关性：合并'))
print(data.corr().round(4).T)
data = data.drop(['平均停留时间'], axis=1)

**************************计算相关性：合并**************************
         日均UV   平均注册率   平均搜索量    访问深度   订单转化率   投放总时间
日均UV   1.0000 -0.0512 -0.0735 -0.0217 -0.0452 -0.0385
平均注册率 -0.0512  1.0000  0.2381  0.1063  0.3166 -0.0141
平均搜索量 -0.0735  0.2381  1.0000  0.0631  0.1259 -0.0268
访问深度  -0.0217  0.1063  0.0631  1.0000  0.1637  0.0575
订单转化率 -0.0452  0.3166  0.1259  0.1637  1.0000 -0.0046
投放总时间 -0.0385 -0.0141 -0.0268  0.0575 -0.0046  1.0000


C:\Users\吕强\AppData\Local\Temp\ipykernel_28776\3066185843.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(data.corr().round(4).T)


In [28]:
# 数据标准化：归一化Min-Max，0-1区间
# SKLEAN-preprocessing-minmax
from sklearn.preprocessing import MinMaxScaler
matrix = data.iloc[:,1:7]
min_max_model = MinMaxScaler()
data_rescaled = min_max_model.fit_transform(matrix)
print(data_rescaled.round(2))


[[0.   0.18 0.02 0.01 0.12 0.66]
 [0.01 0.1  0.03 0.01 0.01 0.62]
 [0.   0.06 0.05 0.01 0.01 0.1 ]
 ...
 [0.01 0.01 0.   0.   0.   0.72]
 [0.05 0.   0.   0.   0.   0.31]
 [0.   0.   0.   0.53 0.   0.62]]
